In [85]:
from alpha_vantage.timeseries import TimeSeries
import pandas as pd
import numpy as np
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import math, random
from datetime import datetime
import datetime as dt
import yfinance as yf
import tweepy
import preprocessor as p
import re
from sklearn.linear_model import LinearRegression
from textblob import TextBlob
import nltk

import constants as ct
import Tweet as tweet

In [86]:
  def get_historical(quote):
        end = datetime.now()
        start = datetime(end.year-2,end.month,end.day)
        data = yf.download(quote, start=start, end=end)
        df = pd.DataFrame(data=data)
        df.to_csv(''+quote+'.csv')
        if(df.empty):
            ts = TimeSeries(key='N6A6QT6IBFJOPJ70',output_format='pandas')
            data, meta_data = ts.get_daily_adjusted(symbol='NSE:'+quote, outputsize='full')
            #Format df
            #Last 2 yrs rows => 502, in ascending order => ::-1
            data=data.head(503).iloc[::-1]
            data=data.reset_index()
            #Keep Required cols only
            df=pd.DataFrame()
            df['Date']=data['date']
            df['Open']=data['1. open']
            df['High']=data['2. high']
            df['Low']=data['3. low']
            df['Close']=data['4. close']
            df['Adj Close']=data['5. adjusted close']
            df['Volume']=data['6. volume']
            df.to_csv(''+quote+'.csv',index=False)
        return

In [87]:
get_historical("AAPL")

[*********************100%***********************]  1 of 1 completed


In [88]:
def retrieving_tweets_polarity(symbol):
    stock_ticker_map = pd.read_csv('Yahoo-Finance-Ticker-Symbols.csv')
    stock_full_form = stock_ticker_map[stock_ticker_map['Ticker']==symbol]
    symbol = stock_full_form['Name'].to_list()[0][0:12]

    auth = tweepy.OAuthHandler(ct.consumer_key, ct.consumer_secret)
    auth.set_access_token(ct.access_token, ct.access_token_secret)
    user = tweepy.API(auth)
        
    tweets = tweepy.Cursor(user.search_users, q=symbol, tweet_mode='extended', lang='en',exclude_replies=True).items(ct.num_of_tweets)
        
    tweet_list = [] #List of tweets alongside polarity
    global_polarity = 0 #Polarity of all tweets === Sum of polarities of individual tweets
    tw_list=[] #List of tweets only => to be displayed on web page
    #Count Positive, Negative to plot pie chart
    pos=0 #Num of pos tweets
    neg=0 #Num of negative tweets
    print(vars(tweets))

In [89]:
retrieving_tweets_polarity("TSLA")

{'page_iterator': <tweepy.cursor.PageIterator object at 0x000002260316AD10>, 'limit': 300, 'current_page': None, 'page_index': -1, 'num_tweets': 0}


In [90]:
import tweepy

# Fill the X's with the credentials obtained by
# following the above mentioned procedure.
consumer_key = "kPDPSzWGry3dtHvmyayqgG1Es"
consumer_secret = "XntDJ7kJuT7rNHsS1LHuVJXIpBsokWiNUK8ivxDQw1BxKgTPju"
access_key = "1278974128721035265-S59VjeWGj2L1Y35VPPWeWhVKvHNO2W"
access_secret = "ocbd0b8imLll9oXhFCqrMXA0WTI7L54MpDFJpRKLXaRc3"

# Function to extract tweets
def get_tweets(username):
		
		# Authorization to consumer key and consumer secret
		auth = tweepy.OAuthHandler(consumer_key, consumer_secret)

		# Access to user's access key and access secret
		auth.set_access_token(access_key, access_secret)

		# Calling api
		api = tweepy.API(auth)

		# 200 tweets to be extracted
		number_of_tweets=200
		tweets = api.user_timeline(screen_name=username)

		# Empty Array
		tmp=[]

		# create array of tweet information: username,
		# tweet id, date/time, text
		tweets_for_csv = [tweet.text for tweet in tweets] # CSV file created
		for j in tweets_for_csv:

			# Appending tweets to the empty array tmp
			tmp.append(j)

		# Printing the tweets
		print(tmp)


# Driver code
if __name__ == '__main__':

	# Here goes the twitter handle for the user
	# whose tweets are to be extracted.
	get_tweets("TSLA")


Forbidden: 403 Forbidden
453 - You currently have Essential access which includes access to Twitter API v2 endpoints only. If you need access to this endpoint, you’ll need to apply for Elevated access via the Developer Portal. You can learn more here: https://developer.twitter.com/en/docs/twitter-api/getting-started/about-twitter-api#v2-access-leve

In [93]:
import snscrape.modules.twitter as sntwitter
import pandas as pd

query = "(from:AAPL) until:2023-04-17 since:2010-01-01"
tweets = []
limit = 500

# Iterate through Twitter search results using snscrape module
for i, tweet in enumerate(sntwitter.TwitterSearchScraper(query).get_items()):
    if i >= limit:
        break
    tweets.append({
    'Date': tweet.date,
    'User': tweet.user.username,
    'Tweet': tweet.content
    })

# Convert list of dictionaries to pandas dataframe
df = pd.DataFrame(tweets)
print(df)

# Save dataframe to csv
df.to_csv('tweets1.csv', index=False)

C:\Users\soham\AppData\Local\Temp\ipykernel_18184\305603488.py:15: DeprecatedFeatureWarning: content is deprecated, use rawContent instead
  'Tweet': tweet.content


                         Date  User             Tweet
0   2013-06-11 20:00:03+00:00  aapl  $438.71 (--0.18)
1   2013-06-11 18:00:03+00:00  aapl   $440.30 (+1.41)
2   2013-06-11 16:00:03+00:00  aapl   $440.90 (+2.01)
3   2013-06-11 14:00:04+00:00  aapl  $435.40 (--3.49)
4   2013-06-11 13:00:06+00:00  aapl    $438.89 (0.00)
..                        ...   ...               ...
486 2013-01-15 15:00:07+00:00  aapl          $489.682
487 2013-01-15 14:00:02+00:00  aapl           $501.75
488 2013-01-14 21:00:16+00:00  aapl           $500.77
489 2013-01-14 20:15:25+00:00  aapl           $502.92
490 2013-01-13 23:53:59+00:00  aapl           $520.30

[491 rows x 3 columns]
